In [ ]:
import time
import json
import requests
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize

class OpenDotaAPI():
    def __init__(self, verbose = False):
        self.verbose = verbose
        self.last_match_id = 0

    def _call(self, url, parameters, tries= 2):
        for i in range(tries):
            try:
                if self.verbose: print("Sending API request... ", end="", flush=True)
                resp = requests.get(url, params= parameters, timeout= 20)
                load_resp = json.loads(resp.text)
                if self.verbose: print("done")
                return load_resp
            except Exception as e:
                print("failed. Trying again in 5s")
                print(e)
                time.sleep(5)
        else:
            ValueError("Unable to connect to OpenDota API")

    # Return a list of 100 recent matches; save smaller match_id
    def get_recent_matches(self, use_last_match = False):
        params = dict()
        if use_last_match:
            params['less_than_match_id'] = self.last_match_id
            params["mmr_descending"] = 1
        url = "https://api.opendota.com/api/publicMatches"
        matches = self._call(url, params)
        #self.last_match_id = min([item['match_id'] for item in matches])
        return matches

def main():
    api = OpenDotaAPI(verbose= True)
    # recent_matches = filter_matches(api.get_recent_matches(use_last_match = True))
    recent_matches = api.get_recent_matches(use_last_match = True)
    return recent_matches


def filter_matches(matches_list):
    return list(filter(lambda m: _filter_function(m), matches_list))

def _filter_function(match):
    if match['duration'] < 1000 or match['duration'] > 4200:
        return False
    elif match['lobby_type'] < 5 or match['lobby_type'] > 7:
        return False
    else:
        return True

def execute():
    df = pd.DataFrame()
    for i in range(10):
        temp = json_normalize(main())
        df = pd.concat([df,temp], ignore_index=True)
        print("Sleeping......")
        time.sleep(600)
    return df

In [ ]:
df = execute()

Sending API request... done
Sending API request... done
Sending API request... done


In [ ]:
df.to_csv("Dataset_dota2.csv", index=False)